<a href="https://colab.research.google.com/github/Kozhedu/Car_Moldova/blob/main/%D0%94%D0%97_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn import preprocessing as prep 

from sklearn.metrics import f1_score

from sklearn.preprocessing import MinMaxScaler

from sklearn.compose import make_column_transformer

from sklearn.pipeline import make_pipeline

from sklearn.metrics import r2_score

from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
from sklearn.utils.fixes import loguniform

from sklearn.model_selection import ShuffleSplit

from sklearn.linear_model import Lasso

# 1. ОПТИМИЗАЦИЯ ЛИНЕЙНОЙ РЕГРЕССИИ

## 1.1. Загружаем чистый набор данных с  новыми признаками

In [ ]:
df = pd.read_csv("cars_moldova_categor.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32474 entries, 0 to 32473
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Make                  32474 non-null  object 
 1   Model                 32474 non-null  object 
 2   Year                  32474 non-null  int64  
 3   Style                 32474 non-null  object 
 4   Distance              32474 non-null  float64
 5   Engine_capacity(cm3)  32474 non-null  float64
 6   Fuel_type             32474 non-null  object 
 7   Transmission          32474 non-null  object 
 8   Price(euro)           32474 non-null  float64
 9   Age                   32474 non-null  int64  
 10  km_year               32474 non-null  float64
 11  km_year_q             32474 non-null  object 
 12  Make_val              32474 non-null  int64  
 13  Price_mean_make       32474 non-null  float64
dtypes: float64(5), int64(3), object(6)
memory usage: 3.5+ MB


In [ ]:
df.head(2)

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro),Age,km_year,km_year_q,Make_val,Price_mean_make
0,Toyota,Prius,2011,Hatchback,195000.0,1800.0,Hybrid,Automatic,7750.0,11,17727.272727,high_mileage,3091,9629.435781
1,Renault,Grand Scenic,2014,Universal,135000.0,1500.0,Diesel,Manual,8550.0,8,16875.000000,average_mileage,2371,6624.550822


In [ ]:
data = df.drop(columns = ['Price(euro)'])
target = df['Price(euro)']

In [ ]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['Make', 'Model', 'Style', 'Fuel_type', 'Transmission', 'km_year_q'] 
 len =  6
numerical columns:	  ['Year', 'Distance', 'Engine_capacity(cm3)', 'Price(euro)', 'Age', 'km_year', 'Make_val', 'Price_mean_make'] 
 len =  8


In [ ]:
df_num = df.copy()

In [ ]:
df_num

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro),Age,km_year,km_year_q,Make_val,Price_mean_make
0,Toyota,Prius,2011,Hatchback,195000.0,1800.0,Hybrid,Automatic,7750.0,11,17727.272727,high_mileage,3091,9629.435781
1,Renault,Grand Scenic,2014,Universal,135000.0,1500.0,Diesel,Manual,8550.0,8,16875.000000,average_mileage,2371,6624.550822
2,Renault,Laguna,2012,Universal,110000.0,1500.0,Diesel,Manual,6550.0,10,11000.000000,little_mileage,2371,6624.550822
3,Opel,Astra,2006,Universal,200000.0,1600.0,Metan/Propan,Manual,4100.0,16,12500.000000,average_mileage,1867,4169.550080
4,Mercedes,Vito,2000,Microvan,300000.0,2200.0,Diesel,Manual,3490.0,22,13636.363636,average_mileage,3032,12960.053100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32469,Volkswagen,Passat,2016,Sedan,88000.0,1800.0,Petrol,Automatic,11500.0,6,14666.666667,average_mileage,3450,6839.976522
32470,Land Rover,Freelander,2002,Crossover,225000.0,1800.0,Metan/Propan,Manual,4400.0,20,11250.000000,little_mileage,288,22631.614583
32471,Dacia,Logan Mcv,2015,Universal,89000.0,1500.0,Diesel,Manual,7000.0,7,12714.285714,average_mileage,1348,5327.074926
32472,Mazda,6,2006,Combi,370000.0,2000.0,Diesel,Manual,4000.0,16,23125.000000,high_mileage,444,5145.808559


## 1.2. Делаем предобработку с помощью Pipeline и One-Hot Encoding.

In [ ]:
X = df_num.drop('Price(euro)',axis=1)
y = df_num['Price(euro)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
ohe = OneHotEncoder()
ohe.fit(X[['Make', 'Model', 'Style', 'Fuel_type', 'Transmission', 'km_year_q']])

OneHotEncoder()

In [ ]:
ct = make_column_transformer(
    (OneHotEncoder(categories = ohe.categories_),['Make', 'Model', 'Style', 'Fuel_type', 'Transmission', 'km_year_q']),
    remainder = 'passthrough'
)

## 1.3. Обучаем простую модель

In [ ]:
pipe = make_pipeline(ct, LinearRegression())

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Alfa Romeo', 'Audi', 'BMW', 'Chevrolet', 'Chrysler', 'Citroen',
       'Dacia', 'Daewoo', 'Daihatsu', 'Dodge', 'Fiat', 'Ford', 'GAZ',
       'Honda', 'Hyundai', 'Infiniti', 'Jaguar', 'Jeep', 'KIA',
       'Land Rover', 'Lexus', 'Lincoln', 'Mazda', 'Mer...
       'Microvan', 'Minivan', 'Pickup', 'Roadster', 'SUV', 'Sedan',
       'Universal'], dtype=object),
                                                                            array(['Diesel', 'Electric', 'Hybrid', 'Metan/Propan', 'Petrol',
       'Plug-in Hybrid'], dtype=object),
                                                                            array(['Automatic', 'Manual'], dtype=object),
                                                                            array(['ave

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:

print(f'R2 Score : {r2_score(y_test, y_pred)*100:.2f}%')

R2 Score : 67.65%


# 2. ОПТИМИЗАЦИЯ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

In [ ]:
pipe = Pipeline([
    ('preprocessors', preprocessors),
    ('model', LogisticRegression(C=0.001, random_state=42))
])

In [ ]:
pipe.fit(X_train, y_train)

Оценим Тренировочную и Валидационную модель

In [ ]:
print(f"Train F1: {calculate_f1_score(pipe, X_train, y_train):.4f}")

print(f"Val F1: {calculate_f1_score(pipe, X_test, y_test):.4f}")